# 2D reconstruction from 1D images

This is the last notebook in the series of computer vision course and this is a definitive guide on 2D reconstruction from 1D images. The ideas we have been building upon will now be compiled in this notebook.

In [1]:
#using the data provided in the course material. It is in 2D, so we will convert it into 1D image and then perform
#rigid tranformation in 2D

import pickle
import numpy as np

def load(filename):
    filename += '.pkl'
    with open(filename, 'rb') as file:
        data = pickle.load(file)
    return data


#this is the image which we are going to reconstruct in the next notebook

image =load('/Users/srtpan/Downloads/data/NoiseFree/NoiseFree')

In [2]:
#The input arrays p_original and q_original are 2×n. Each column contains the canonical Euclidean coordinates 
#of one of the n points in one of the two camera reference systems

p_original=image['p']
q_original=image['q']

The following function uses approximate triangulation to find the 2×n array P of positions of the n 2D points in the coordinate reference system of the first camera that project to the image points in p and q.

We can input the rotation R and translation R from the function we had created in the previous notebook. We will also calculate Q which is the real-world point but seen from the reference system of the second camera. 

In [3]:
def triangulate(p, q, t, R):
    #getting the number of data points
    n=p.shape[1]
    
    #creating an empty 2D matrix with n data points
    P = np.zeros((2, n))
    
    #getting the i and j components of the rotation matrix
    i, j = R[0], R[1]
    
    #creating a projecting matrix with the second component of rotation matrix
    kt = np.dot(j, t)
    proj = np.dot(i, t)
    
    C = np.array(((1, 0), (0, 1), (0, 0))).astype(float)
    c = np.zeros(3)
    
    for m in range(n):
        C[:2, 1] = -p[:2, m]
        C[1:, :] = np.outer(q[:2, m], j) - proj
        c[2:] = kt * q[:1, m] - proj
        
        #finding the optimum solution using least squares method
        x = np.linalg.lstsq(C, c, rcond=None)
        P[:, m] = x[0]
    
    #creating the point 
    Q = np.dot(R, P - np.outer(t, np.ones((1, n))))
    return P, Q

In [4]:
def estimate_rotation_translation(p,q):
    
    #calculating the number of data points
    n=p_original.flatten().shape[0]
    
    #reshaping the first camera projection from 2D to 1D
    p=p_original.reshape((1,n))
    
    #reshaping the second camera projection from 2D to 1D
    q=q_original.reshape((1,n))
    
    #creating a 1D array to project the data from 1D to 2D
    o = np.ones((1, n)).astype(float)
    
    #concatentaing extra row to make p and q two-dimensional
    p, q = np.concatenate((p, o)), np.concatenate((q, o))
    A = np.zeros((n, 4))
    for k in range(n):
        A[k, :] = np.outer(q[:, k], p[:, k]).flatten() 
        
    #using SVD to create essential matrix  
    _, _, VT = np.linalg.svd(A)
    E = np.reshape(VT[-1, :], (2, 2))
    U, sigma, V = np.linalg.svd(E)
    W=np.array([[0,-1],[1,0]])
    dot_1=np.dot(U,W.T)
    
    #getting the rotation matrix
    R=np.dot(dot_1,V)
    dot1=np.dot(U,W)
    dot2=np.dot(dot1,sigma)
    
    #getting the translation matrix
    t=np.dot(dot2,U.T)
    
    #use of triangulation to reconstruct the original point for which we had two 1D projections, p and q
    P,Q=triangulate(p, q, t, R)
    
    return t,R,P,Q

In [5]:
t,R,P,Q=estimate_rotation_translation(p_original,q_original)

In [6]:
P.shape

(2, 230)

In [7]:
Q.shape

(2, 230)

In [8]:
P

array([[-0.13217329, -0.13086516, -0.12893773, -0.12621049, -0.12245114,
        -0.11736334, -0.1390845 , -0.13912184, -0.13873007, -0.13774415,
        -0.13593718, -0.13299684, -0.1439704 , -0.14542053, -0.14670868,
        -0.14772037, -0.14828335, -0.14813706, -0.14571305, -0.14846994,
        -0.15138362, -0.15442788, -0.1575451 , -0.16062162, -0.14289674,
        -0.14658048, -0.15072544, -0.15541557, -0.1607519 , -0.1668531 ,
        -0.1338224 , -0.13766992, -0.14215003, -0.14743223, -0.15375228,
        -0.16144786,  0.04100707,  0.04891248,  0.05775621,  0.06770097,
         0.07894701,  0.09174309,  0.02069008,  0.02783131,  0.03589676,
         0.04506237,  0.05554883,  0.06763605, -0.00189603,  0.00418681,
         0.01113579,  0.01913386,  0.028416  ,  0.03928808, -0.02697711,
        -0.02233639, -0.01695924, -0.01067055, -0.00323865,  0.0056483 ,
        -0.05467289, -0.05195842, -0.04874935, -0.04490982, -0.0402528 ,
        -0.03451478, -0.08481824, -0.08459237, -0.0

In [9]:
Q

array([[-0.54367779, -0.54534196, -0.54768348, -0.55089531, -0.55522522,
        -0.56098866, -0.53575847, -0.53599816, -0.53671714, -0.53809285,
        -0.5403683 , -0.54387692, -0.52987834, -0.52861553, -0.52755259,
        -0.52681475, -0.52658886, -0.52715569, -0.5271949 , -0.52453049,
        -0.52173107, -0.51882997, -0.51589505, -0.51305492, -0.52917531,
        -0.52549375, -0.52135491, -0.51667687, -0.51136181, -0.50529623,
        -0.53758091, -0.53366378, -0.52910271, -0.52372521, -0.51729137,
        -0.50945763, -0.71376413, -0.72177003, -0.73072969, -0.74080867,
        -0.75221066, -0.76518867, -0.69314658, -0.70037019, -0.7085329 ,
        -0.71781376, -0.72843737, -0.7406886 , -0.67023847, -0.67637929,
        -0.68339938, -0.69148497, -0.70087526, -0.71188166, -0.64482066,
        -0.64948721, -0.65489981, -0.66123655, -0.66873317, -0.67770712,
        -0.61678946, -0.61948917, -0.62268689, -0.62652032, -0.63117933,
        -0.63693161, -0.58634467, -0.58650867, -0.5

## References:

1. Duke University CS 527 Spring'22: https://courses.cs.duke.edu//spring22/compsci527/notes/n_10_reconstruction.pdf
2. Wikipedia triangulation: https://en.wikipedia.org/wiki/Triangulation_(computer_vision)